# New notebook

In [9]:
import pandas as pd
from sqlalchemy import create_engine
db_name = "test1.db"
engine = create_engine(f"sqlite:///{db_name}")

In [10]:
my_dict = {
    'class':['Five','Six','Three'],
    'No':[5,2,3]
}

In [11]:
df = pd.DataFrame(my_dict)
df

,class,No
0,Five,5
1,Six,2
2,Three,3


In [12]:
df.to_sql('student', engine)

3